<a href="https://colab.research.google.com/github/srushtibhavsar/Gambling/blob/main/WorkingHiWiPipeline_given_sys_mess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


In [24]:
from datasets import load_metric

In [9]:
bleu = load_metric("bleu")

<ipython-input-9-def88d6c8e4e>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("bleu")


In [4]:
from transformers import AutoTokenizer
import transformers
import torch

In [5]:
model = "srushtibhavsar/FineTuneWithGivenSysMess"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model = model,
    torch_dtype = torch.float16,
    device_map = "auto"
)


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
sys_mess = """
You are an assistant that is given a list of change graphs in an edge format. That is, the graph is given edge by edge. The graphs are directed, labeled graphs. An edge is serialized as
"e src_id tgt_id edge_label src_label tgt_label"

Labels are dictionaries. If a node appears in more than one edge, the second time it appears it is replaced by "_" to avoid repetition.

E.g.:
e 0 1 a b bar
e 1 2 bla _ foo

The second edge here would be equivalent to:
"e 1 2 bla bar foo"

There are some change graphs given as examples. Graphs are separated by "\n\n$$\n---\n".

The last graph in this list of graphs is not yet complete. Exactly one edge is missing.
Your task is it, to complete the last graph by guessing the last edge. You can guess this typically by looking at the examples and trying to deduce the patterns in the examples. Give this missing edge in the format
"e src_id tgt_id edge_label src_label tgt_label". Note that the beginning "e" is already part of the prompt.
"""

In [22]:
prompt = "t # 23 e 2 0 \"{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'details'}\" \"{'changeType': 'Add', 'type': 'object', 'className': 'EAnnotation', 'attributes': {'id': '_LMTmQt6tEei97MD7GK1RmA', 'source':'http://www.eclipse.org/emf/2002/GenModel','details':['org.eclipse.emf.ecore.impl.EStringToStringMapEntryImpl@472f2ab4 (key: documentation, value: \\nSpecifies the marker to be used for displaying the end data point on the line in the chart.)'],'eModelElement':'endMarker'}}\" \"{'changeType': 'Add', 'type': 'object', 'className': 'EStringToStringMapEntry', 'attributes': {'id': '_LMTmQ96tEei97MD7GK1RmA', 'key':'documentation','value':'\\nSpecifies the marker to be used for displaying the end data point on the line in the chart.'}}\" e 4 2 \"{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eAnnotations'}\" \"{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_I-pb3t6tEei97MD7GK1RmA', 'eAnnotations':['org.eclipse.emf.ecore.impl.EAnnotationImpl@8f398b (source: http://www.eclipse.org/emf/2002/GenModel)','org.eclipse.emf.ecore.impl.EAnnotationImpl@2606b21 (source: http:///org/eclipse/emf/ecore/util/ExtendedMetaData)'],'name':'endMarker','ordered':'true','unique':'true','lowerBound':'1','upperBound':'1','many':'false','required':'true','eType':'Marker','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@e927959 (expression: Marker)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'GanttSeries','containment':'true','container':'false','resolveProxies':'false','eReferenceType':'Marker'}}\" _ e 4 3 \"{'changeType': 'Remove', 'type': 'reference', 'referenceTypeName': 'eAnnotations'}\" _ \"{'changeType': 'Remove', 'type': 'object', 'className': 'EAnnotation', 'attributes': {'id': '_I-pb396tEei97MD7GK1RmA', 'source':'http://www.eclipse.org/emf/2002/GenModel','details':['org.eclipse.emf.ecore.impl.EStringToStringMapEntryImpl@1bac7cc5 (key: documentation, value: \\r\\nSpecifies the marker to be used for displaying the end data point on the line in the chart.)'],'eModelElement':'endMarker'}}\" e"
sequence = pipeline(
    f"<s>[INST]<<SYS>> {sys_mess} <</SYS>> {prompt} [/INST]",
    do_sample = True,
    top_k = 10,
    eos_token_id = tokenizer.eos_token_id,
    max_length = 2000,
)
for seq in sequence:
  print(f"Result: {seq['generated_text']}")

Result: <s>[INST]<<SYS>> 
You are an assistant that is given a list of change graphs in an edge format. That is, the graph is given edge by edge. The graphs are directed, labeled graphs. An edge is serialized as
"e src_id tgt_id edge_label src_label tgt_label"

Labels are dictionaries. If a node appears in more than one edge, the second time it appears it is replaced by "_" to avoid repetition.

E.g.:
e 0 1 a b bar
e 1 2 bla _ foo

The second edge here would be equivalent to:
"e 1 2 bla bar foo"

There are some change graphs given as examples. Graphs are separated by "

$$
---
".

The last graph in this list of graphs is not yet complete. Exactly one edge is missing.
Your task is it, to complete the last graph by guessing the last edge. You can guess this typically by looking at the examples and trying to deduce the patterns in the examples. Give this missing edge in the format
"e src_id tgt_id edge_label src_label tgt_label". Note that the beginning "e" is already part of the prompt.


In [25]:
print(sequence[0]['generated_text'])

<s>[INST]<<SYS>> 
You are an assistant that is given a list of change graphs in an edge format. That is, the graph is given edge by edge. The graphs are directed, labeled graphs. An edge is serialized as
"e src_id tgt_id edge_label src_label tgt_label"

Labels are dictionaries. If a node appears in more than one edge, the second time it appears it is replaced by "_" to avoid repetition.

E.g.:
e 0 1 a b bar
e 1 2 bla _ foo

The second edge here would be equivalent to:
"e 1 2 bla bar foo"

There are some change graphs given as examples. Graphs are separated by "

$$
---
".

The last graph in this list of graphs is not yet complete. Exactly one edge is missing.
Your task is it, to complete the last graph by guessing the last edge. You can guess this typically by looking at the examples and trying to deduce the patterns in the examples. Give this missing edge in the format
"e src_id tgt_id edge_label src_label tgt_label". Note that the beginning "e" is already part of the prompt.
 <</SYS>

In [27]:
# Your generated text
generated_text = sequence[0]['generated_text']
# Tokenize the generated text
tokenized_text = tokenizer(generated_text, truncation=True)

# Print the tokenized text
#print("Original Text:", generated_text)
print("Tokenized Text:", tokenized_text)

Tokenized Text: {'input_ids': [1, 1, 29961, 25580, 29962, 9314, 14816, 29903, 6778, 29871, 13, 3492, 526, 385, 20255, 393, 338, 2183, 263, 1051, 310, 1735, 18445, 297, 385, 7636, 3402, 29889, 2193, 338, 29892, 278, 3983, 338, 2183, 7636, 491, 7636, 29889, 450, 18445, 526, 10624, 29892, 301, 24025, 18445, 29889, 530, 7636, 338, 7797, 1891, 408, 13, 29908, 29872, 4765, 29918, 333, 260, 4141, 29918, 333, 7636, 29918, 1643, 4765, 29918, 1643, 260, 4141, 29918, 1643, 29908, 13, 13, 4775, 29879, 526, 21503, 4314, 29889, 960, 263, 2943, 5692, 297, 901, 1135, 697, 7636, 29892, 278, 1473, 931, 372, 5692, 372, 338, 8611, 491, 11119, 29908, 304, 4772, 21159, 654, 29889, 13, 13, 29923, 29889, 29887, 4898, 13, 29872, 29871, 29900, 29871, 29896, 263, 289, 2594, 13, 29872, 29871, 29896, 29871, 29906, 12995, 903, 7953, 13, 13, 1576, 1473, 7636, 1244, 723, 367, 7126, 304, 29901, 13, 29908, 29872, 29871, 29896, 29871, 29906, 12995, 2594, 7953, 29908, 13, 13, 8439, 526, 777, 1735, 18445, 2183, 408, 6455,

In [32]:
print("Tokenized Text:", tokenized_text["input_ids"])

Tokenized Text: [1, 1, 29961, 25580, 29962, 9314, 14816, 29903, 6778, 29871, 13, 3492, 526, 385, 20255, 393, 338, 2183, 263, 1051, 310, 1735, 18445, 297, 385, 7636, 3402, 29889, 2193, 338, 29892, 278, 3983, 338, 2183, 7636, 491, 7636, 29889, 450, 18445, 526, 10624, 29892, 301, 24025, 18445, 29889, 530, 7636, 338, 7797, 1891, 408, 13, 29908, 29872, 4765, 29918, 333, 260, 4141, 29918, 333, 7636, 29918, 1643, 4765, 29918, 1643, 260, 4141, 29918, 1643, 29908, 13, 13, 4775, 29879, 526, 21503, 4314, 29889, 960, 263, 2943, 5692, 297, 901, 1135, 697, 7636, 29892, 278, 1473, 931, 372, 5692, 372, 338, 8611, 491, 11119, 29908, 304, 4772, 21159, 654, 29889, 13, 13, 29923, 29889, 29887, 4898, 13, 29872, 29871, 29900, 29871, 29896, 263, 289, 2594, 13, 29872, 29871, 29896, 29871, 29906, 12995, 903, 7953, 13, 13, 1576, 1473, 7636, 1244, 723, 367, 7126, 304, 29901, 13, 29908, 29872, 29871, 29896, 29871, 29906, 12995, 2594, 7953, 29908, 13, 13, 8439, 526, 777, 1735, 18445, 2183, 408, 6455, 29889, 12367,

In [10]:
# The instruction dataset to use
dataset_name = "srushtibhavsar/HiwiDatasetWithGivenSystemMessage"

In [11]:
from datasets import load_dataset

In [28]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [13]:
dataset

Dataset({
    features: ['text'],
    num_rows: 215
})

In [ ]:
dataset[2]

In [35]:

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

# Tokenize the 'text' column in your dataset
tokenized_dataset = dataset.map(
    lambda examples: tokenizer(examples['text'], truncation=True),
    batched=True,
)

# Check the tokens in the first example
#print("Original Text:", dataset['text'][2])
print("Tokenized Text:", tokenized_dataset['input_ids'][2])


Tokenized Text: [1, 1, 29961, 25580, 29962, 3532, 14816, 29903, 6778, 13, 3492, 526, 385, 20255, 393, 338, 2183, 263, 1051, 310, 1735, 18445, 297, 385, 7636, 3402, 29889, 2193, 338, 29892, 278, 3983, 338, 2183, 7636, 491, 7636, 29889, 450, 18445, 526, 10624, 29892, 301, 24025, 18445, 29889, 530, 7636, 338, 7797, 1891, 408, 13, 29908, 29872, 4765, 29918, 333, 260, 4141, 29918, 333, 7636, 29918, 1643, 4765, 29918, 1643, 260, 4141, 29918, 1643, 29908, 13, 13, 4775, 29879, 526, 21503, 4314, 29889, 960, 263, 2943, 5692, 297, 901, 1135, 697, 7636, 29892, 278, 1473, 931, 372, 5692, 372, 338, 8611, 491, 11119, 29908, 304, 4772, 21159, 654, 29889, 29871, 13, 13, 29923, 29889, 29887, 4898, 13, 29872, 29871, 29900, 29871, 29896, 263, 289, 2594, 13, 29872, 29871, 29896, 29871, 29906, 12995, 903, 7953, 13, 13, 1576, 1473, 7636, 1244, 723, 367, 7126, 304, 29901, 13, 29908, 29872, 29871, 29896, 29871, 29906, 12995, 2594, 7953, 29908, 13, 13, 8439, 526, 777, 1735, 18445, 2183, 408, 6455, 29889, 12367,

In [36]:
# Assuming tokenized_text is the output from tokenizer
tokenText = tokenized_text['input_ids']

# Print the input_ids
print("Input IDs:", tokenText)

Input IDs: [1, 1, 29961, 25580, 29962, 9314, 14816, 29903, 6778, 29871, 13, 3492, 526, 385, 20255, 393, 338, 2183, 263, 1051, 310, 1735, 18445, 297, 385, 7636, 3402, 29889, 2193, 338, 29892, 278, 3983, 338, 2183, 7636, 491, 7636, 29889, 450, 18445, 526, 10624, 29892, 301, 24025, 18445, 29889, 530, 7636, 338, 7797, 1891, 408, 13, 29908, 29872, 4765, 29918, 333, 260, 4141, 29918, 333, 7636, 29918, 1643, 4765, 29918, 1643, 260, 4141, 29918, 1643, 29908, 13, 13, 4775, 29879, 526, 21503, 4314, 29889, 960, 263, 2943, 5692, 297, 901, 1135, 697, 7636, 29892, 278, 1473, 931, 372, 5692, 372, 338, 8611, 491, 11119, 29908, 304, 4772, 21159, 654, 29889, 13, 13, 29923, 29889, 29887, 4898, 13, 29872, 29871, 29900, 29871, 29896, 263, 289, 2594, 13, 29872, 29871, 29896, 29871, 29906, 12995, 903, 7953, 13, 13, 1576, 1473, 7636, 1244, 723, 367, 7126, 304, 29901, 13, 29908, 29872, 29871, 29896, 29871, 29906, 12995, 2594, 7953, 29908, 13, 13, 8439, 526, 777, 1735, 18445, 2183, 408, 6455, 29889, 12367, 2987

In [43]:
len(tokenText)

1197

In [44]:
len(tokenized_dataset['input_ids'][2])

1152

In [45]:
bleu.compute(predictions = tokenText, references = tokenized_dataset['input_ids'][2])

TypeError: ignored

In [40]:
pretrained_model = "NousResearch/Llama-2-7b-chat-hf"

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
pretrained_pipeline = transformers.pipeline(
    "text-generation",
    model = pretrained_model,
    torch_dtype = torch.float16,
    device_map = "auto"
)


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [41]:
prompt = "t # 23 e 2 0 \"{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'details'}\" \"{'changeType': 'Add', 'type': 'object', 'className': 'EAnnotation', 'attributes': {'id': '_LMTmQt6tEei97MD7GK1RmA', 'source':'http://www.eclipse.org/emf/2002/GenModel','details':['org.eclipse.emf.ecore.impl.EStringToStringMapEntryImpl@472f2ab4 (key: documentation, value: \\nSpecifies the marker to be used for displaying the end data point on the line in the chart.)'],'eModelElement':'endMarker'}}\" \"{'changeType': 'Add', 'type': 'object', 'className': 'EStringToStringMapEntry', 'attributes': {'id': '_LMTmQ96tEei97MD7GK1RmA', 'key':'documentation','value':'\\nSpecifies the marker to be used for displaying the end data point on the line in the chart.'}}\" e 4 2 \"{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eAnnotations'}\" \"{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_I-pb3t6tEei97MD7GK1RmA', 'eAnnotations':['org.eclipse.emf.ecore.impl.EAnnotationImpl@8f398b (source: http://www.eclipse.org/emf/2002/GenModel)','org.eclipse.emf.ecore.impl.EAnnotationImpl@2606b21 (source: http:///org/eclipse/emf/ecore/util/ExtendedMetaData)'],'name':'endMarker','ordered':'true','unique':'true','lowerBound':'1','upperBound':'1','many':'false','required':'true','eType':'Marker','eGenericType':'org.eclipse.emf.ecore.impl.EGenericTypeImpl@e927959 (expression: Marker)','changeable':'true','volatile':'false','transient':'false','unsettable':'false','derived':'false','eContainingClass':'GanttSeries','containment':'true','container':'false','resolveProxies':'false','eReferenceType':'Marker'}}\" _ e 4 3 \"{'changeType': 'Remove', 'type': 'reference', 'referenceTypeName': 'eAnnotations'}\" _ \"{'changeType': 'Remove', 'type': 'object', 'className': 'EAnnotation', 'attributes': {'id': '_I-pb396tEei97MD7GK1RmA', 'source':'http://www.eclipse.org/emf/2002/GenModel','details':['org.eclipse.emf.ecore.impl.EStringToStringMapEntryImpl@1bac7cc5 (key: documentation, value: \\r\\nSpecifies the marker to be used for displaying the end data point on the line in the chart.)'],'eModelElement':'endMarker'}}\" e"
sequence = pretrained_pipeline(
    f"<s>[INST] {prompt} [/INST]",
    do_sample = True,
    top_k = 10,
    eos_token_id = pretrained_tokenizer.eos_token_id,
    max_length = 2000,
)
for seq in sequence:
  print(f"Result: {seq['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Result: <s>[INST] t # 23 e 2 0 "{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'details'}" "{'changeType': 'Add', 'type': 'object', 'className': 'EAnnotation', 'attributes': {'id': '_LMTmQt6tEei97MD7GK1RmA', 'source':'http://www.eclipse.org/emf/2002/GenModel','details':['org.eclipse.emf.ecore.impl.EStringToStringMapEntryImpl@472f2ab4 (key: documentation, value: \nSpecifies the marker to be used for displaying the end data point on the line in the chart.)'],'eModelElement':'endMarker'}}" "{'changeType': 'Add', 'type': 'object', 'className': 'EStringToStringMapEntry', 'attributes': {'id': '_LMTmQ96tEei97MD7GK1RmA', 'key':'documentation','value':'\nSpecifies the marker to be used for displaying the end data point on the line in the chart.'}}" e 4 2 "{'changeType': 'Add', 'type': 'reference', 'referenceTypeName': 'eAnnotations'}" "{'changeType': 'Preserve', 'type': 'object', 'className': 'EReference', 'attributes': {'id': '_I-pb3t6tEei97MD7GK1RmA', 'eAnnotations':['org.ecl